In [2]:
import pandas as pd
import numpy as np
csv = pd.read_csv('export/15s_133features_filter_std_pca8.csv')
print(f"{csv.shape=}")

def getXY(csv, do_print=True):
    df = csv.copy()
    y_ori = np.array(df['type'])
    y_ori[y_ori == 0] = 0
    y_ori[y_ori == 2] = 1
    y_ori
    head = list(df.keys())
    head.remove('Unnamed: 0')
    head.remove('name')
    head.remove('score')
    head.remove('type')
    X_ori = np.array(df[head])
    if(do_print):
        print(f"{X_ori.shape=} {y_ori.shape=}")
    return X_ori, y_ori

csv.shape=(720, 12)


In [3]:
def train_model(X_ori,y_ori, do_print=True):
    X,y = X_ori.copy(), y_ori.copy()

    from sklearn.svm import SVC
    from sklearn.utils import shuffle
    from sklearn.model_selection import cross_val_score

    X_shuff,y_shuff = shuffle(X,y)
    model = SVC(kernel='rbf')
    cross = cross_val_score(model, X_shuff, y_shuff, cv=10)

    model = SVC(kernel='rbf')
    model.fit(X_shuff, y_shuff)
    ans = model.predict(X_shuff)
    acc = sum(ans == y_shuff) / len(y_shuff)
    if(do_print):
        print(acc, cross.mean(), cross)
        return model
    else:
        return model, acc, cross

def report(names,labels,predicts,do_print = True):
    correct = 0
    wrong = 0
    prev_name = None
    prev_label = None
    count = 0

    table = []
    summary = []
    wrong3of4 = 0
    table.append(f"Name\tLabel\tPredict")
    table.append("="*30)
    for name,predict,label in zip(names,predicts,labels):
        if(predict == label): correct += 1
        else: wrong += 1

        text_predict = 'non-st' if predict == 0 else 'stress'
        text_label = 'non-st' if label == 0 else 'stress'
        if(prev_name != None and prev_name != name and count > 0): 
            text_prev_label = 'non-st' if prev_label == 0 else 'stress'
            summary.append(f"label={text_prev_label} | {count} {prev_name}")
            if(count > 5): wrong3of4 += 1
            count = 0
        if(predict != label):
            count = count + 1
            table.append(f"{name}\t{text_label}\t{text_predict}")

        prev_label = label
        prev_name = name
        
    if(do_print):
        print('\n'.join(table))    
        print()
        print("Incorrect summany")
        print("="*30)
        print('\n'.join(summary))
        print(f"Correct={correct} | Wrong={wrong} | Acc={correct / (correct + wrong) * 100}%")
    else:
        acc = correct / (correct + wrong)
        return summary, acc, wrong3of4

In [4]:
# model_all
X,y = getXY(csv.copy())
model_all = train_model(X,y)

samples,labels = getXY(csv.copy())
names = csv['name']
predicts = model_all.predict(samples) 
print()
report(names,labels,predicts)

X_ori.shape=(720, 8) y_ori.shape=(720,)
0.9847222222222223 0.9736111111111112 [0.93055556 0.95833333 0.94444444 0.98611111 1.         1.
 1.         0.95833333 0.95833333 1.        ]
X_ori.shape=(720, 8) y_ori.shape=(720,)

Name	Label	Predict
amp	non-st	stress
beer	non-st	stress
flm	stress	non-st
miiw	non-st	stress
MJ	non-st	stress
nisit	non-st	stress
nisit	non-st	stress
pla	stress	non-st
prin	non-st	stress
prin	non-st	stress
prin	non-st	stress

Incorrect summany
label=non-st | 1 amp
label=non-st | 1 beer
label=stress | 1 flm
label=non-st | 1 miiw
label=non-st | 1 MJ
label=non-st | 2 nisit
label=stress | 1 pla
label=non-st | 3 prin
Correct=709 | Wrong=11 | Acc=98.47222222222223%


In [5]:
non_stress_names = list(set(csv[csv['type'] == 0]['name']))
non_stress_names.sort()
stress_names = list(set(csv[csv['type'] == 2]['name']))
stress_names.sort()
print(f"{non_stress_names=}\n{stress_names=}")

non_stress_names=['MJ', 'amp', 'beau', 'beer', 'int', 'job', 'miiw', 'mu', 'nisit', 'noey', 'nuclear', 'praewphan', 'prin', 'sam', 'shin', 'suyo']
stress_names=['aui', 'bam', 'bank', 'bas', 'cedric', 'dt', 'eiyu', 'fabby', 'fahmai', 'flm', 'gon', 'kao', 'kee', 'minkhant', 'new', 'pang', 'pla', 'ploy', 'yee', 'yong']


In [6]:
# model_filter
for remove_name in non_stress_names:
    selected_csv = csv.copy()
    cond = selected_csv['name'] != remove_name
    
    X,y = getXY(selected_csv[cond],do_print=False)
    model,acc,cross = train_model(X,y,do_print=False)
    # print(f"ACC = {acc*100}% | CV = {cross.mean()*100}%")

    samples,labels = getXY(csv.copy(),do_print=False)
    names = csv['name']
    predicts = model.predict(samples)
    # print()
    summary,acc,_ = report(names,labels,predicts,do_print=False)


    print(f"REMOVE NAME: {remove_name} - ACC = {acc*100}%")
    for text in summary:
        print(f" - {text}")
    # print(" \n".join(summary))
    # print("************************************************************")


REMOVE NAME: MJ - ACC = 98.33333333333333%
 - label=non-st | 1 beer
 - label=stress | 1 flm
 - label=non-st | 1 miiw
 - label=non-st | 3 MJ
 - label=non-st | 1 nisit
 - label=stress | 1 pla
 - label=non-st | 3 prin
 - label=non-st | 1 suyo
REMOVE NAME: amp - ACC = 97.63888888888889%
 - label=non-st | 8 amp
 - label=non-st | 1 beer
 - label=stress | 1 flm
 - label=non-st | 1 miiw
 - label=non-st | 1 MJ
 - label=non-st | 1 nisit
 - label=non-st | 3 prin
 - label=non-st | 1 suyo
REMOVE NAME: beau - ACC = 98.47222222222223%
 - label=non-st | 1 amp
 - label=non-st | 1 beer
 - label=stress | 1 flm
 - label=non-st | 1 miiw
 - label=non-st | 1 MJ
 - label=non-st | 2 nisit
 - label=stress | 1 pla
 - label=non-st | 3 prin
REMOVE NAME: beer - ACC = 98.33333333333333%
 - label=non-st | 1 amp
 - label=non-st | 1 beer
 - label=stress | 1 flm
 - label=non-st | 1 miiw
 - label=non-st | 1 MJ
 - label=non-st | 2 nisit
 - label=stress | 1 pla
 - label=non-st | 3 prin
 - label=non-st | 1 suyo
REMOVE NAME:

In [7]:
# model_filter
for remove_name in stress_names:
    selected_csv = csv.copy()
    cond = selected_csv['name'] != remove_name
    
    X,y = getXY(selected_csv[cond],do_print=False)
    model,acc,cross = train_model(X,y,do_print=False)
    # print(f"ACC = {acc*100}% | CV = {cross.mean()*100}%")

    samples,labels = getXY(csv.copy(),do_print=False)
    names = csv['name']
    predicts = model.predict(samples)
    # print()
    summary,acc,_ = report(names,labels,predicts,do_print=False)


    print(f"REMOVE NAME: {remove_name} - ACC = {acc*100}%")
    for text in summary:
        print(f" - {text}")
    # print(" \n".join(summary))
    # print("************************************************************")


REMOVE NAME: aui - ACC = 95.69444444444444%
 - label=stress | 20 aui
 - label=non-st | 1 beer
 - label=stress | 1 flm
 - label=non-st | 1 miiw
 - label=non-st | 1 MJ
 - label=non-st | 1 nisit
 - label=stress | 1 pla
 - label=non-st | 5 prin
REMOVE NAME: bam - ACC = 95.97222222222223%
 - label=stress | 20 bam
 - label=stress | 1 flm
 - label=non-st | 1 miiw
 - label=non-st | 1 MJ
 - label=non-st | 2 nisit
 - label=non-st | 3 prin
 - label=non-st | 1 suyo
REMOVE NAME: bank - ACC = 95.83333333333334%
 - label=non-st | 1 amp
 - label=stress | 20 bank
 - label=non-st | 1 beer
 - label=stress | 1 flm
 - label=non-st | 1 miiw
 - label=non-st | 1 MJ
 - label=non-st | 1 nisit
 - label=non-st | 3 prin
 - label=non-st | 1 suyo
REMOVE NAME: bas - ACC = 98.33333333333333%
 - label=non-st | 1 amp
 - label=stress | 1 bas
 - label=non-st | 1 beer
 - label=stress | 1 flm
 - label=non-st | 1 miiw
 - label=non-st | 1 MJ
 - label=non-st | 2 nisit
 - label=stress | 1 pla
 - label=non-st | 3 prin
REMOVE NAM

In [8]:
for time_select in range(0,20):
    selected_csv = csv.copy()
    cond = selected_csv['Unnamed: 0'] % 20 == time_select
    # print(selected_csv[cond])

    X,y = getXY(selected_csv[cond],do_print=False)
    model,acc,cross = train_model(X,y,do_print=False)
    # print(f"ACC = {acc*100}% | CV = {cross.mean()*100}%")

    samples,labels = getXY(csv.copy(),do_print=False)
    names = csv['name']
    predicts = model.predict(samples)
    # print()
    summary,acc,wrong3of4 = report(names,labels,predicts,do_print=False)


    print(f"Time Select: {str(time_select).zfill(2)} - ACC = {round(acc*100,2)}% - wrong3of4 = {wrong3of4}/{len(names)//20}")
    # for text in summary:
    #     print(f" - {text}")
    # print(" \n".join(summary))
    # print("************************************************************")



Time Select: 00 - ACC = 82.5% - wrong3of4 = 8/36
Time Select: 01 - ACC = 75.0% - wrong3of4 = 10/36
Time Select: 02 - ACC = 81.53% - wrong3of4 = 9/36
Time Select: 03 - ACC = 79.17% - wrong3of4 = 9/36
Time Select: 04 - ACC = 81.53% - wrong3of4 = 8/36
Time Select: 05 - ACC = 82.78% - wrong3of4 = 8/36
Time Select: 06 - ACC = 83.61% - wrong3of4 = 8/36
Time Select: 07 - ACC = 81.67% - wrong3of4 = 8/36
Time Select: 08 - ACC = 83.47% - wrong3of4 = 6/36
Time Select: 09 - ACC = 84.03% - wrong3of4 = 6/36
Time Select: 10 - ACC = 82.78% - wrong3of4 = 8/36
Time Select: 11 - ACC = 82.22% - wrong3of4 = 7/36
Time Select: 12 - ACC = 83.61% - wrong3of4 = 6/36
Time Select: 13 - ACC = 83.19% - wrong3of4 = 8/36
Time Select: 14 - ACC = 82.64% - wrong3of4 = 8/36
Time Select: 15 - ACC = 82.22% - wrong3of4 = 7/36
Time Select: 16 - ACC = 80.69% - wrong3of4 = 8/36
Time Select: 17 - ACC = 79.03% - wrong3of4 = 11/36
Time Select: 18 - ACC = 80.28% - wrong3of4 = 9/36
Time Select: 19 - ACC = 77.22% - wrong3of4 = 10/36

In [9]:
# comb_list

In [10]:
from itertools import combinations

comb_list = []
# for time_frame in range(1,20):
comb_list.extend(list(combinations(range(20),2)))
# print(comb_list)


for time_selects in comb_list:
    selected_csv = csv.copy()
    cond1 = selected_csv['Unnamed: 0'] % 20 == time_selects[0]
    cond2 = selected_csv['Unnamed: 0'] % 20 == time_selects[1]
    cond = cond1 | cond2
    # print(time_selects)
    # print(selected_csv[cond])

    X,y = getXY(selected_csv[cond],do_print=False)
    model,acc,cross = train_model(X,y,do_print=False)
    # print(f"ACC = {acc*100}% | CV = {cross.mean()*100}%")

    samples,labels = getXY(csv.copy(),do_print=False)
    names = csv['name']
    predicts = model.predict(samples)
    # print()
    summary,acc,wrong3of4 = report(names,labels,predicts,do_print=False)

    if(acc > 0.92 ):
        print(f"Time Select: {str(time_selects[0]).zfill(2)} {str(time_selects[1]).zfill(2)} - ACC = {round(acc*100,2)}% - wrong3of4 = {wrong3of4}/{len(names)//20}")
        for text in summary:
            print(f" - {text}")
        # print(" \n".join(summary))
    # print("************************************************************")



Time Select: 05 06 - ACC = 92.08% - wrong3of4 = 3/36
 - label=stress | 5 aui
 - label=non-st | 1 beer
 - label=non-st | 1 job
 - label=non-st | 4 miiw
 - label=non-st | 2 MJ
 - label=non-st | 4 nisit
 - label=non-st | 11 praewphan
 - label=non-st | 19 prin
 - label=non-st | 4 sam
 - label=non-st | 6 suyo
Time Select: 05 13 - ACC = 92.92% - wrong3of4 = 2/36
 - label=stress | 5 aui
 - label=stress | 1 bam
 - label=non-st | 1 beer
 - label=stress | 2 flm
 - label=non-st | 5 job
 - label=non-st | 4 miiw
 - label=non-st | 1 MJ
 - label=non-st | 4 nisit
 - label=non-st | 7 praewphan
 - label=non-st | 17 prin
 - label=non-st | 1 sam
 - label=non-st | 3 suyo


In [13]:
from itertools import combinations

comb_list = []
# for time_frame in range(1,20):
comb_list.extend(list(combinations(range(20),3)))
# print(comb_list)


for time_selects in comb_list:
    selected_csv = csv.copy()
    cond1 = selected_csv['Unnamed: 0'] % 20 == time_selects[0]
    cond2 = selected_csv['Unnamed: 0'] % 20 == time_selects[1]
    cond3 = selected_csv['Unnamed: 0'] % 20 == time_selects[2]
    cond = cond1 | cond2 | cond3
    # print(time_selects)
    # print(selected_csv[cond])

    X,y = getXY(selected_csv[cond],do_print=False)
    model,acc,cross = train_model(X,y,do_print=False)
    # print(f"ACC = {acc*100}% | CV = {cross.mean()*100}%")

    samples,labels = getXY(csv.copy(),do_print=False)
    names = csv['name']
    predicts = model.predict(samples)
    # print()
    summary,acc,wrong3of4 = report(names,labels,predicts,do_print=False)

    if(acc > 0.95 ):
        print(f"Time Select: {str(time_selects[0]).zfill(2)} {str(time_selects[1]).zfill(2)} {str(time_selects[2]).zfill(2)} - ACC = {round(acc*100,2)}% - wrong3of4 = {wrong3of4}/{len(names)//20}")
        for text in summary:
            print(f" - {text}")
        # print(" \n".join(summary))
    # print("************************************************************")



Time Select: 05 06 13 - ACC = 95.42% - wrong3of4 = 1/36
 - label=stress | 1 aui
 - label=stress | 1 bam
 - label=non-st | 1 beer
 - label=stress | 1 flm
 - label=non-st | 1 miiw
 - label=non-st | 1 MJ
 - label=non-st | 4 nisit
 - label=non-st | 2 praewphan
 - label=non-st | 17 prin
 - label=non-st | 1 sam
 - label=non-st | 3 suyo
